<a href="https://colab.research.google.com/github/Iamjohnko/Data-science-Project-Portfolio/blob/main/ANALYSIS_OF_SALES_WITH_KEY_METRICS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import time, warnings
import datetime as dt
warnings.filterwarnings("ignore")

In [ ]:
# Read the data
df=pd.read_csv("Sale_Consumption.csv")


df.head()

,Inovice_Date,Customer_ID,Customer,Invoice_no,Quantity,Rate,Monetary_Value,Destination,Truck no
0,01/06/2022,ABEN001,ABENG TIMOTHY,SQ001385,60000,169,"10,110,000.00",OGOJA,SKP715
1,01/06/2022,ANY001,ANYANWU OBIOMA,SQ001377,30000,163,"4,890,000.00",MBAISE,AAA981
2,01/06/2022,BENDOSKY001,BENDOSKY GLOBAL LTD,SQ001375,15000,163,"2,445,000.00",MBAISE,AAA974
3,01/06/2022,DIC003,DICKSON JOHN,SQ001444,15000,161,"2,407,500.00",PHC,SKP711
4,01/06/2022,KONAKRIKON01,KONAKRIKON,SQ001378,15000,162,"2,430,000.00",UYO,SKP721


In [ ]:
#converting the type of Invoice Date Field from string to datetime.
df['Inovice_Date'] = pd.to_datetime(df['Inovice_Date'], format= '%d/%m/%Y')

In [ ]:
#creating YearMonth field for the ease of reporting and visualization
df['InvoiceYearMonth'] = df['Inovice_Date'].map(lambda date: 100*date.year + date.month)

In [ ]:
#calculate Revenue for each row and create a new dataframe with YearMonth - Revenue columns
df['Revenue'] = df['Rate'] * df['Quantity']
df_revenue = df.groupby(['InvoiceYearMonth'])['Revenue'].sum().reset_index()
df_revenue

,InvoiceYearMonth,Revenue
0,202201,138705000
1,202202,249825000
2,202203,145455000
3,202204,231838000
4,202205,156980000
5,202206,846147000
6,202207,1316474500
7,202208,775240500
8,202209,156131000
9,202210,189525000


In [ ]:
import plotly as py
import plotly.offline as pyoff
import plotly.graph_objs as go

In [ ]:
#X and Y axis inputs for Plotly graph. We use Scatter for line graphs
plot_data = [
    go.Scatter(
        x=df_revenue['InvoiceYearMonth'],
        y=df_revenue['Revenue'],
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Montly Revenue'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [ ]:
#using pct_change() function to see monthly percentage change
df_revenue['MonthlyGrowth'] = df_revenue['Revenue'].pct_change()

#showing first 5 rows
df_revenue.head()

#visualization - line graph
plot_data = [
    go.Scatter(
        x=df_revenue.query("InvoiceYearMonth < 202207")['InvoiceYearMonth'],
        y=df_revenue.query("InvoiceYearMonth < 202207")['MonthlyGrowth'],
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Montly Growth Rate'
    )

fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [ ]:
#creating a new dataframe with UK customers only
tx_uk = df.query("Destination=='ABA'").reset_index(drop=True)

#creating monthly active customers dataframe by counting unique Customer IDs
tx_monthly_active = tx_uk.groupby('InvoiceYearMonth')['Customer_ID'].nunique().reset_index()

#print the dataframe
tx_monthly_active

#plotting the output
plot_data = [
    go.Bar(
        x=tx_monthly_active['InvoiceYearMonth'],
        y=tx_monthly_active['Customer_ID'],
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Monthly Active Customers'
    )

fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [ ]:
tx_monthly_active

,InvoiceYearMonth,Customer_ID
0,202201,2
1,202202,10
2,202203,6
3,202204,4
4,202205,3
5,202206,9
6,202207,18
7,202208,16
8,202209,6
9,202210,5


In [ ]:
#create a new dataframe for no. of order by using quantity field
tx_monthly_sales = tx_uk.groupby('InvoiceYearMonth')['Quantity'].sum().reset_index()

#print the dataframe
tx_monthly_sales

#plot
plot_data = [
    go.Bar(
        x=tx_monthly_sales['InvoiceYearMonth'],
        y=tx_monthly_sales['Quantity'],
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Monthly Total # of Order'
    )

fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [ ]:
# create a new dataframe for average revenue by taking the mean of it
tx_monthly_order_avg = tx_uk.groupby('InvoiceYearMonth')['Revenue'].mean().reset_index()

#print the dataframe
tx_monthly_order_avg

#plot the bar chart
plot_data = [
    go.Bar(
        x=tx_monthly_order_avg['InvoiceYearMonth'],
        y=tx_monthly_order_avg['Revenue'],
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Monthly Order Average'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [ ]:
#print the dataframe
tx_monthly_order_avg

,InvoiceYearMonth,Revenue
0,202201,3.742500e+06
1,202202,6.254000e+06
2,202203,3.957857e+06
3,202204,3.006429e+06
4,202205,4.375000e+06
5,202206,4.629889e+06
6,202207,4.749722e+06
7,202208,4.435385e+06
8,202209,4.910000e+06
9,202210,4.245000e+06


In [ ]:
tx_min_purchase

,Customer_ID,MinPurchaseDate,MinPurchaseYearMonth
0,ALBA001,2022-03-08,202203
1,AUSM001,2022-02-06,202202
2,BRI001,2022-04-08,202204
3,CHAB001,2022-07-29,202207
4,CHIEMELA001,2022-02-06,202202
5,CHU002,2022-02-06,202202
6,CYM001,2022-02-08,202202
7,DITH001,2022-07-27,202207
8,DUR001,2022-07-28,202207
9,EKEN001,2022-07-22,202207


In [ ]:

tx_uk.head()

,Inovice_Date,Customer_ID,Customer,Invoice_no,Quantity,Rate,Monetary_Value,Destination,Truck no,InvoiceYearMonth,Revenue,MinPurchaseDate,MinPurchaseYearMonth,UserType
0,2022-01-06,MAS001,MASTA PETROLEUM,SQ001376,30000,162,"4,845,000.00",ABA,SKP708,202201,4860000,2022-01-06,202201,New
1,2022-06-21,MAS001,MASTA PETROLEUM,SQ001546,30000,169,"5,070,000.00",ABA,AAA970,202206,5070000,2022-01-06,202201,Existing
2,2022-05-07,MAS001,MASTA PETROLEUM,SQ001629,30000,175,"5,250,000.00",ABA,BWR917,202205,5250000,2022-01-06,202201,Existing
3,2022-08-16,MAS001,MASTA PETROLEUM,SQ002246,30000,178,"5,340,000.00",ABA,UNKNOW,202208,5340000,2022-01-06,202201,Existing
4,2022-02-06,AUSM001,AUSMONTEE,SQ001389,15000,162,"2,422,500.00",ABA,AAA978,202202,2430000,2022-02-06,202202,New


In [ ]:
tx_user_type_revenue

,InvoiceYearMonth,UserType,Revenue
0,202201,New,7485000
1,202202,Existing,15165000
2,202202,New,78645000
3,202203,Existing,2430000
4,202203,New,25275000
5,202204,Existing,15795000
6,202204,New,5250000
7,202205,Existing,7875000
8,202205,New,5250000
11,202207,Existing,137660000


In [ ]:
#create a dataframe contaning CustomerID and first purchase date
tx_min_purchase = tx_uk.groupby('Customer_ID').Inovice_Date.min().reset_index()
tx_min_purchase.columns = ['Customer_ID','MinPurchaseDate']
tx_min_purchase['MinPurchaseYearMonth'] = tx_min_purchase['MinPurchaseDate'].map(lambda date: 100*date.year + date.month)

#merge first purchase date column to our main dataframe (tx_uk)
tx_uk = pd.merge(tx_uk, tx_min_purchase, on='Customer_ID')

tx_uk.head()

#create a column called User Type and assign Existing
#if User's First Purchase Year Month before the selected Invoice Year Month
tx_uk['UserType'] = 'New'
tx_uk.loc[tx_uk['InvoiceYearMonth']>tx_uk['MinPurchaseYearMonth'],'UserType'] = 'Existing'

#calculate the Revenue per month for each user type
tx_user_type_revenue = tx_uk.groupby(['InvoiceYearMonth','UserType'])['Revenue'].sum().reset_index()

#filtering the dates and plot the result
tx_user_type_revenue = tx_user_type_revenue.query("InvoiceYearMonth != 202206 and InvoiceYearMonth != 202208")
plot_data = [
    go.Scatter(
        x=tx_user_type_revenue.query("UserType == 'Existing'")['InvoiceYearMonth'],
        y=tx_user_type_revenue.query("UserType == 'Existing'")['Revenue'],
        name = 'Existing'
    ),
    go.Scatter(
        x=tx_user_type_revenue.query("UserType == 'New'")['InvoiceYearMonth'],
        y=tx_user_type_revenue.query("UserType == 'New'")['Revenue'],
        name = 'New'
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='New vs Existing'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [ ]:
#create a dataframe that shows new user ratio - we also need to drop NA values (first month new user ratio is 0)
tx_user_ratio = tx_uk.query("UserType == 'New'").groupby(['InvoiceYearMonth'])['Customer_ID'].nunique()/tx_uk.query("UserType == 'Existing'").groupby(['InvoiceYearMonth'])['Customer_ID'].nunique()
tx_user_ratio = tx_user_ratio.reset_index()
tx_user_ratio = tx_user_ratio.dropna()

#print the dafaframe
tx_user_ratio

#plot the result

plot_data = [
    go.Bar(
        x=tx_user_ratio.query("InvoiceYearMonth>201101 and InvoiceYearMonth<202208")['InvoiceYearMonth'],
        y=tx_user_ratio.query("InvoiceYearMonth>201101 and InvoiceYearMonth<202208")['Customer_ID'],
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='New Customer Ratio'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [ ]:
#identify which users are active by looking at their revenue per month
tx_user_purchase = tx_uk.groupby(['Customer_ID','InvoiceYearMonth'])['Revenue'].sum().reset_index()

#create retention matrix with crosstab
tx_retention = pd.crosstab(tx_user_purchase['Customer_ID'], tx_user_purchase['InvoiceYearMonth']).reset_index()

tx_retention.head()

#create an array of dictionary which keeps Retained & Total User count for each month
months = tx_retention.columns[2:]
retention_array = []
for i in range(len(months)-1):
    retention_data = {}
    selected_month = months[i+1]
    prev_month = months[i]
    retention_data['InvoiceYearMonth'] = int(selected_month)
    retention_data['TotalUserCount'] = tx_retention[selected_month].sum()
    retention_data['RetainedUserCount'] = tx_retention[(tx_retention[selected_month]>0) & (tx_retention[prev_month]>0)][selected_month].sum()
    retention_array.append(retention_data)

#convert the array to dataframe and calculate Retention Rate
tx_retention = pd.DataFrame(retention_array)
tx_retention['RetentionRate'] = tx_retention['RetainedUserCount']/tx_retention['TotalUserCount']

#plot the retention rate graph
plot_data = [
    go.Scatter(
        x=tx_retention.query("InvoiceYearMonth<202208")['InvoiceYearMonth'],
        y=tx_retention.query("InvoiceYearMonth<202208")['RetentionRate'],
        name="organic"
    )

]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Monthly Retention Rate'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [ ]:
tx_user_type_revenue.query('InvoiceYearMonth != 202206 and InvoiceYearMonth != 202208')

,InvoiceYearMonth,UserType,Revenue
0,202201,New,7485000
1,202202,Existing,15165000
2,202202,New,78645000
3,202203,Existing,2430000
4,202203,New,25275000
5,202204,Existing,15795000
6,202204,New,5250000
7,202205,Existing,7875000
8,202205,New,5250000
11,202207,Existing,137660000


In [ ]:
tx_retention

,InvoiceYearMonth,TotalUserCount,RetainedUserCount,RetentionRate
0,202203,6,1,0.166667
1,202204,4,1,0.250000
2,202205,3,1,0.333333
3,202206,9,3,0.333333
4,202207,18,5,0.277778
5,202208,16,9,0.562500
6,202209,6,5,0.833333
7,202210,5,2,0.400000
8,202211,3,0,0.000000
9,202212,2,0,0.000000


In [ ]:
tx_user_purchase = tx_uk.groupby(['Customer_ID','InvoiceYearMonth'])['Revenue'].sum().astype(int).reset_index()

In [ ]:
tx_user_purchase.head()

,Customer_ID,InvoiceYearMonth,Revenue
0,ALBA001,202203,7875000
1,ALBA001,202207,7875000
2,AUSM001,202202,5055000
3,AUSM001,202207,6140000
4,AUSM001,202208,7980000
